In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np
import pygame
from datetime import datetime

from tf_agents.environments import py_environment
# from tf_agents.environments import tf_environment
# from tf_agents.environments import tf_py_environment
# from tf_agents.environments import utils
# from tf_agents.specs import array_spec
# from tf_agents.environments import wrappers
# from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

# tf.compat.v1.enable_v2_behavior()

import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
# from simple_dqn_torch_2020 import Agent
# from utils import plotLearning
from sklearn.model_selection import learning_curve

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# class CardGameEnv(py_environment.PyEnvironment):

#   def __init__(self):
#     self._action_spec = array_spec.BoundedArraySpec(
#         shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
#     self._observation_spec = array_spec.BoundedArraySpec(
#         shape=(1,), dtype=np.int32, minimum=0, name='observation')
#     self._state = 0
#     self._episode_ended = False

#   def action_spec(self):
#     return self._action_spec

#   def observation_spec(self):
#     return self._observation_spec

#   def _reset(self):
#     self._state = 0
#     self._episode_ended = False
#     return ts.restart(np.array([self._state], dtype=np.int32))

#   def _step(self, action):

#     if self._episode_ended:
#       # The last action ended the episode. Ignore the current action and start
#       # a new episode.
#       return self.reset()

#     # Make sure episodes don't go on forever.
#     if action == 1:
#       self._episode_ended = True
#     elif action == 0:
#       new_card = np.random.randint(1, 11)
#       self._state += new_card
#     else:
#       raise ValueError('`action` should be 0 or 1.')

#     if self._episode_ended or self._state >= 21:
#       reward = self._state - 21 if self._state <= 21 else -21
#       return ts.termination(np.array([self._state], dtype=np.int32), reward)
#     else:
#       return ts.transition(
#           np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0)
# env = gym.make('LunarLander-v2')
env = gym.make('Copy-v0')
env

<TimeLimit<CopyEnv<Copy-v0>>>

In [3]:
#set up game area layout
# By changing the block_size other aspects will scale
block_size = 30    # Size of side of square
blocks_w = 10      # game width (in blocks)
blocks_h = 20      # game height (in blocks)
border_w = 500
border_h = 100
end_wait = 2000

play_w = blocks_w * block_size  # Width is 10 blocks
play_h = blocks_h * block_size  # Height is 20 blocks
full_w = 2 * border_w + play_w
full_h = 2 * border_h + play_h + 50
top_left_x = border_w
top_left_y = 2 * border_h
start_x = int(blocks_w / 2)

NO_KEY = 0
LEFT_KEY = 1
RIGHT_KEY = 2
DOWN_KEY = 3
UP_KEY = 4


I = [['..0..',
      '..0..',
      '..0..',
      '..0..',
      '.....'],
     ['.....',
      '0000.',
      '.....',
      '.....',
      '.....']]
#J
J = [['.....',
      '.0...',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..00.',
      '..0..',
      '..0..',
      '.....'],
     ['.....',
      '.....',
      '.000.',
      '...0.',
      '.....'],
     ['.....',
      '..0..',
      '..0..',
      '.00..',
      '.....']]
#L
L = [['.....',
      '.....',
      '.000.',
      '.0...',
      '.....'],
     ['.....',
      '.00..',
      '..0..',
      '..0..',
      '.....'],
     ['.....',
      '...0.',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..0..',
      '..0..',
      '..00.',
      '.....']]
#O
O = [['.....',
      '.....',
      '.00..',
      '.00..',
      '.....']]
#S
S = [['.....',
      '..0..',
      '..00.',
      '...0.',
      '.....'],
     ['.....',
      '.....',
      '..00.',
      '.00..',
      '.....']]
#T
T = [['.....',
      '..0..',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..0..',
      '..00.',
      '..0..',
      '.....'],
     ['.....',
      '.....',
      '.000.',
      '..0..',
      '.....'],
     ['.....',
      '..0..',
      '.00..',
      '..0..',
      '.....']]
#Z
Z = [['.....',
      '.....',
      '.00..',
      '..00.',
      '.....'],
     ['.....',
      '..0..',
      '.00..',
      '.0...',
      '.....']]
shapes = [I, J, L, O, S, T, Z]
shapes_color = [(255,0,0),(0,255,0),
                (0,0,255),(0,255,255),
                (255,0,255),(255,255,0),
                (128,0,128)]
pygame.font.init()

In [4]:
class Piece(object):
    def __init__ (self, x, y, shape):
        self.x = x
        self.y = y
        self.shape = shape
        self.color = shapes_color[shapes.index(shape)]
        self.rotation = int(np.random.rand()*len(shape))

def create_grid(locked_pos = {}):
    grid = [[(0,0,0) for _ in range(blocks_w)] for _ in range(blocks_h)]
    for hgt in range(blocks_h):
        for wid in range(blocks_w):
            if (wid, hgt) in locked_pos:
                clr = locked_pos[(wid,hgt)]
                grid[hgt][wid] = clr
    return grid

def convert_shape_fmt(this_shape):
    posit = []
    fmt = this_shape.shape [this_shape.rotation % len(this_shape.shape)]
    for i, line in enumerate(fmt):
        row = list(line)
        for j, col in enumerate(row):
            if col == '0':
                posit.append((int(this_shape.x + j), int(this_shape.y + i)))
                             
    for i, pos in enumerate(posit):
        posit[i] = (int(pos[0]-2), int(pos[1]-4))
    return posit

def valid_space(grid, piece):
    accepted_pos = [[(j,i) for j in range (blocks_w) if grid[i][j] == (0,0,0)] for i in range (blocks_h)]
    accepted_pos = [j for sub in accepted_pos for j in sub]
    formatted = convert_shape_fmt(piece)
    for pos in formatted:
        if pos not in accepted_pos:
            if pos[1] > -1:
                return False
    return True

def check_lost(positions):
    for pos in positions:
        x, y = pos
        if y < -1:
            return True
    return False

def get_shape():
    return Piece(start_x, 0, np.random.choice(shapes))

def draw_text_middle(surface, text, size, color):
    font = pygame.font.SysFont('Arial',size, bold = True)
    bolded = font.render(text, 1, color)
    surface.blit(bolded,((full_w-bolded.get_width())/2, (full_h-bolded.get_height())/2))

def draw_grid(surface, grid):
    for hgt in range(blocks_h+1):
        pygame.draw.line(surface,(128,128,128), 
                         (top_left_x, top_left_y + hgt*block_size), 
                         (top_left_x + blocks_w*block_size, top_left_y + hgt*block_size))
    for wid in range(blocks_w+1):
        pygame.draw.line(surface,(128,128,128), 
                         (top_left_x + wid*block_size, top_left_y), 
                         (top_left_x + wid*block_size, top_left_y + blocks_h*block_size))


def clear_rows(grid, locked):

    inc = 0
    # backward (bottom to top scan)
    for i in range(len(grid)-1, -1, -1):
        row = grid[i]
        if (0, 0, 0) not in row:
            inc += 1
            for j in range(len(row)):
                del locked[(j,i)]
        else:
            if inc > 0:
                k = i+inc
                for j in range(len(row)):
                    if (j,i) in locked:
                        locked[(j,k)] = locked[(j,i)]
                        del locked[(j,i)]

    return inc, locked

def draw_next_shape(surface, shp):
    font = pygame.font.SysFont('Arial',60)
    next_s = font.render('Next Shape:',1, (255,255,255))
    next_x = top_left_x + play_w + 150
    next_y = top_left_y + play_h/2 - 150
    fmt = shp.shape[shp.rotation]
    
    for i, line in enumerate(fmt):
        row = list(line)
        for j,col in enumerate(row):
            if col == '0':
                pygame.draw.rect(surface, shp.color, (next_x + j*block_size, 
                                 next_y + i*block_size, 
                                 block_size, block_size), 0)
    surface.blit(next_s, (next_x - 50, next_y - 100))

def add_score(score):
    now = datetime.now()
    dt_string = now.strftime("%Y/%m/%d %H:%M:%S\n")
    with open('data/score_log','a') as f:
        f.writelines(str(int(score))+','+dt_string)
    return score

def draw_window(surface, grid, score):
    surface.fill((0,0,0))
    pygame.font.init()
    font = pygame.font.SysFont('Arial',60)
    title = font.render('Tetris',1, (255,255,255))
    surface.blit(title,(full_w/2 - title.get_width()/2, 20))
    
    font = pygame.font.SysFont('Arial',60)
    score_s = font.render('Score:',1, (255,255,255))
    score_x = 50
    score_y = top_left_y + 50
    surface.blit(score_s, (score_x, score_y))
    score_s = font.render(str(score),1, (255,255,255))
    score_x = 100
    score_y += 100
    surface.blit(score_s, (score_x, score_y))
    
    for hgt in range(blocks_h):
        for wid in range(blocks_w):
            pygame.draw.rect(surface, grid[hgt][wid],
                             (top_left_x + wid*block_size, 
                            top_left_y + hgt*block_size, 
                            block_size, block_size), 0)
    pygame.draw.rect(surface, (255,0,0), (top_left_x, top_left_y, play_w, play_h), 4)
    draw_grid(surface, grid)
#     pygame.display.update()

In [5]:
class Tetris(py_environment.PyEnvironment):

    def __init__(self):
#         self._action_spec = array_spec.BoundedArraySpec(
#             shape=(), dtype=np.int32, minimum=0, maximum=4, name='action')
#         self._observation_spec = array_spec.BoundedArraySpec(
#             shape=(blocks_h,blocks_w), dtype=np.int32, minimum=0, name='observation')
        self._state = 0
        self._episode_ended = False
        self.win = pygame.display.set_mode((full_w, full_h))
        pygame.display.set_caption('Tetris')
        self.win.fill((0, 0, 0))
#         draw_text_middle(win,"Press any key to Start", 60, (255, 255, 255))
        self.locked_blocks = {}
        self.grid = create_grid(self.locked_blocks)

        self.change_piece = False
#     run = True
        self.next_piece = get_shape()
        self.curr_piece = get_shape()
        self.score = 0
        pygame.display.update()
    
#     clock = pygame.time.Clock()
#     fall_time = 0
#     fall_speed =.27
#     level_time = 0
#     score = 0

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self.grid
#         return self._observation_spec

    def time_step_spec(self):
        """Return time_step_spec."""

    def _reset(self):
        self._state = 0
        self._episode_ended = False
        return ts.restart(np.array([self._state], dtype=np.int32))

    def _step(self, action):

#     if self._episode_ended:
#       # The last action ended the episode. Ignore the current action and start
#       # a new episode.
#       TETRIS BOARD CLOGGED
#       return self.reset()

#     # Make sure episodes don't go on forever.     
#     TETRIS GAME WILL END
#     if action == 1:
#       self._episode_ended = True
#     elif action == 0:
#       new_card = np.random.randint(1, 11)
#       self._state += new_card
#     else:
        if action < 0 or action > 4:
            raise ValueError('`action` should be between 0 and 4.')
        self.curr_piece.y += 1
        self.grid = create_grid(self.locked_blocks)
        if not (valid_space(self.grid, self.curr_piece)) and (self.curr_piece.y > 0):
            self.curr_piece.y -= 1
            self.change_piece = True
        
        self.grid = create_grid(self.locked_blocks)
        key_choice = action #(self.grid, self.curr_piece)
        if key_choice == LEFT_KEY:
            self.curr_piece.x -= 1
            if not (valid_space(self.grid, self.curr_piece)):
                self.curr_piece.x += 1
        if key_choice == RIGHT_KEY:
            self.curr_piece.x += 1
            if not (valid_space(self.grid, self.curr_piece)):
                self.curr_piece.x -= 1
        if key_choice == DOWN_KEY:
            while valid_space(self.grid, self.curr_piece):
                self.curr_piece.y += 1
            self.curr_piece.y -= 1
        if key_choice == UP_KEY:
            self.curr_piece.rotation += 1 
            self.curr_piece.rotation %= len(self.curr_piece.shape)
            if not (valid_space(self.grid, self.curr_piece)):
                if self.curr_piece.rotation == 0:
                    self.curr_piece.rotation = len(self.curr_piece.shape) - 1
                else:
                    self.curr_piece.rotation -= 1
        # if none of the above, piece drops one row

        shape_pos = convert_shape_fmt(self.curr_piece)
        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                self.grid[y][x] = self.curr_piece.color
        if self.change_piece:
            for pos in shape_pos:
                p = (pos[0],pos[1])
                self.locked_blocks[p] = self.curr_piece.color
            self.curr_piece = self.next_piece
            self.next_piece = get_shape()
            self.change_piece = False
            increment, new_block_set = clear_rows(self.grid, self.locked_blocks)
            self.score += increment * 10
            self.locked_blocks = new_block_set

        draw_window(self.win, self.grid, self.score)
        draw_next_shape(self.win, self.next_piece)
        pygame.display.update()

        self._state = 1
        if check_lost(self.locked_blocks):
#             draw_text_middle(win, "You Lost!", 80, (255, 255, 255))
            self._state = 2
            pygame.display.update()
#             pygame.time.delay(end_wait)
#             run = False
            self.score = add_score(self.score)
            self._episode_ended = True
        if self._episode_ended:
            return ts.termination(np.array([self._state], dtype=np.int32), self.score)
        else:
            return ts.transition(
                np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0)
#     return win, score, grid, locked_blocks, curr_piece, next_piece, check_lost
#     if self._episode_ended or self._state >= 21:
#       reward = self._state - 21 if self._state <= 21 else -21
#       return ts.termination(np.array([self._state], dtype=np.int32), reward)
#     else:
#       return ts.transition(
#           np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0)

In [6]:
# action = np.array(0, dtype=np.int32)  # test with do nothing
# time_step = environment.reset()
# print(time_step)
# while not time_step.is_last():
#   time_step = environment.step(action)
#   print(time_step)


In [7]:
environment = Tetris()
# utils.validate_py_environment(environment, episodes=5)

In [8]:
get_new_card_action = np.array(0, dtype=np.int32)
end_round_action = np.array(2, dtype=np.int32)

# environment = Tetris()
time_step = environment.reset()
# print(time_step)
cumulative_reward = time_step.reward

for episode in range (3):
    time_step = environment.reset()
    done = False
    # for _ in range(20):
    while not done:
        time_step = environment.step(get_new_card_action)
        #     print(time_step)
        cumulative_reward += time_step.reward
        if time_step.step_type == 2:
            print(time_step)
            done = True
            time_step = environment.step(end_round_action)
            
            now = datetime.now()
            dt_string = now.strftime("%Y/%m/%d %H:%M:%S\n")
            with open('data/score_log','a') as f:
                f.writelines(str(episode+1)+','+str(int(time_step.reward))+','+dt_string)
#         return score


#comment out the following line to display last result
pygame.display.quit()

# print(time_step)
cumulative_reward += time_step.reward
print('Final Reward = ', cumulative_reward)

TimeStep(step_type=array(2, dtype=int32), reward=array(0., dtype=float32), discount=array(0., dtype=float32), observation=array([2], dtype=int32))
TimeStep(step_type=array(2, dtype=int32), reward=array(0., dtype=float32), discount=array(0., dtype=float32), observation=array([2], dtype=int32))
TimeStep(step_type=array(2, dtype=int32), reward=array(0., dtype=float32), discount=array(0., dtype=float32), observation=array([2], dtype=int32))
Final Reward =  0.0


In [9]:
class DeepQNetwork(nn.Module):
    def __init__ (self, lr, input_dims, fc1_dims, fc2_dims, n_actions):
        super(DeepQNetwork, self).__init__()
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.fc3 = nn.Linear(self.fc2_dims, self_n_actions)
        self.optimize = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        self.device = T.device('cuda:0' if T.cuda.is_available else 'cpu')
        self.to(self.device)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = F.relu(self.fc3(x))
        return actions

In [15]:
class Agent():
    def __init__ (self, gamma, epsilon, lr, input_dims, batch_size, n_actions, 
                  max_mem_size=100000, eps_end = .01, eps_dec=5e-4):
        self.gamma = gamma
        self.epsilon = epsilon
        self.lr = lr
        self.batch_size = batch_size
        self.eps_end = eps_end
        self.eps_dec = eps_dec
        self.action_space = [i for i in range (n_actions)]
        self.max_mem_size = max_mem_size
        self.mem_cntr = 0
        
        self.Q_eval = DeepQNetwork(self.lr, n_actions = n_actions, input_dims = input_dims,
                                  fc1_dims=256, fc2_dims=256)
        self.state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.bool)
        
    def store_transition(self, state, action, reward, state_, done):
        indx = self.mem_cntr % self.mem_size
        self.state_memory[indx] = state
        self.new_state_memory[indx] = state_
        self.action_memory[indx] = action
        self.reward_memory[indx] = reward
        self.terminal_memory[indx] = done
        self.mem_cntr += 1
        
    def choose_action(self, observation):
        if np.random.random() > self.epsilon:
            state = T.tensor([observation]).to(self.Q_eval.device)
            actions = self.Q_eval_forward(state)
            action = T.argmax(actions).item()
        else:
            action = np.random.choice(self.action_space)
        return action
    
    def learn(self):
        if self.mem_cntr < sel.batch_size:
            return
        self.Q_eval.optimizer.zero.grad()
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, self.batch_size, replace = False)
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        state_batch = T.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        new_state_batch = T.tensor(self.new_state_memory[batch]).to(self.Q_eval.device)
        reward_batch = T.tensor(self.reward_memory[batch]).to(self.Q_eval.device)
        terminal_batch = T.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)
        action_batch = self.action_memory[batch]
        q_eval = self.Q_eval.forward(state_batch)[batch_index, action_batch]
        q_next = self.Q_eval.forward(new_state_batch)
        q_next[terminal_batch] = 0.0
        q_target = reward_batch + self.gamma * T.max(q_next, dim=1)[0]
        loss = self.Q_eval.loss(q_target, q_eval).to(self.Q_eval.device)
        loss.backward()
        self.Q_eval.optimizer.step()
        self.epsilon = self.epsilon - self.eps_dec if self.epsilon>self.eps_min else self.eps_min
        

SyntaxError: invalid syntax (<ipython-input-15-a2ee0be087bf>, line 2)

In [16]:
if __name__ == '__main__':
    env = Tetris()
    agent = Agent(gamma =.99, epsilon = 1.0, batch_size=50, n_actions=5, eps_end = .01, input_dims[blocks_w][blocks_h], lr =.003)
    scores, eps_history = [], []
    n_games = 500
    
    for i in range (n_games):
        score = 0
        done = False
        observation = env.reset()
        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score = reward
            agent.store_transition(observation, action, reward, observation_, done)
            agent.learn()
            observation = onbservation_
        scores.append(score)
        eps_history.append(agent.epsilon)
        avg_score = np.mean(scores[-100:])
        
        print('episode', i, 'score %.2f' % score,
             'average score %.2f' % avg_score,
             'epsilon %.2f' % agent.epsilon)
    x = [i+1 for i in range(n_games)]
    filename = 'data/pytouch_img.png'
    plot_learning_curve(x, scores, eps_history, filename)

SyntaxError: positional argument follows keyword argument (<ipython-input-16-b98d5737b87f>, line 3)

In [24]:
class NeuralNetwork(nn.Module):

    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.number_of_actions = 2
        self.gamma = 0.99
        self.final_epsilon = 0.0001
        self.initial_epsilon = 0.1
        self.number_of_iterations = 2000000
        self.replay_memory_size = 10000
        self.minibatch_size = 32

        self.conv1 = nn.Conv2d(4, 32, 8, 4)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(32, 64, 4, 2)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(64, 64, 3, 1)
        self.relu3 = nn.ReLU(inplace=True)
        self.fc4 = nn.Linear(3136, 512)
        self.relu4 = nn.ReLU(inplace=True)
        self.fc5 = nn.Linear(512, self.number_of_actions)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.conv3(out)
        out = self.relu3(out)
        out = out.view(out.size()[0], -1)
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.fc5(out)

        return out

optimizer = optim.Adam(model.parameters(), lr=1e-6)
criterion = nn.MSELoss()

# create the game
game_state = Tetris()

replay_memory = []


while iteration < model.number_of_iterations:
    # get output from the neural network
    output = model(state)[0]

    # initialize action
    action = torch.zeros([model.number_of_actions], dtype=torch.float32)
    if torch.cuda.is_available():  # put on GPU if CUDA is available
        action = action.cuda()

    # epsilon greedy exploration
    random_action = random.random() <= epsilon
    if random_action:
        print("Performed random action!")
    action_index = [torch.randint(model.number_of_actions, torch.Size([]), dtype=torch.int)
                    if random_action
                    else torch.argmax(output)][0]

    if torch.cuda.is_available():  # put on GPU if CUDA is available
        action_index = action_index.cuda()

    action[action_index] = 1

    # get next state and reward
    image_data_1, reward, terminal = game_state.frame_step(action)
    image_data_1 = resize_and_bgr2gray(image_data_1)
    image_data_1 = image_to_tensor(image_data_1)
    state_1 = torch.cat((state.squeeze(0)[1:, :, :], image_data_1)).unsqueeze(0)

    action = action.unsqueeze(0)
    reward = torch.from_numpy(np.array([reward], dtype=np.float32)).unsqueeze(0)

    # save transition to replay memory
    replay_memory.append((state, action, reward, state_1, terminal))

    # if replay memory is full, remove the oldest transition
    if len(replay_memory) > model.replay_memory_size:
        replay_memory.pop(0)

    # epsilon annealing
    epsilon = epsilon_decrements[iteration]

    # sample random minibatch
    minibatch = random.sample(replay_memory, min(len(replay_memory), model.minibatch_size))

    # unpack minibatch
    state_batch = torch.cat(tuple(d[0] for d in minibatch))
    action_batch = torch.cat(tuple(d[1] for d in minibatch))
    reward_batch = torch.cat(tuple(d[2] for d in minibatch))
    state_1_batch = torch.cat(tuple(d[3] for d in minibatch))

    if torch.cuda.is_available():  # put on GPU if CUDA is available
        state_batch = state_batch.cuda()
        action_batch = action_batch.cuda()
        reward_batch = reward_batch.cuda()
        state_1_batch = state_1_batch.cuda()

    # get output for the next state
    output_1_batch = model(state_1_batch)

    # set y_j to r_j for terminal state, otherwise to r_j + gamma*max(Q)
    y_batch = torch.cat(tuple(reward_batch[i] if minibatch[i][4]
                              else reward_batch[i] + model.gamma * torch.max(output_1_batch[i])
                              for i in range(len(minibatch))))

    # extract Q-value
    q_value = torch.sum(model(state_batch) * action_batch, dim=1)

    # PyTorch accumulates gradients by default, so they need to be reset in each pass
    optimizer.zero_grad()

    # returns a new Tensor, detached from the current graph, the result will never require gradient
    y_batch = y_batch.detach()

    # calculate loss
    loss = criterion(q_value, y_batch)

    # do backward pass
    loss.backward()
    optimizer.step()

    # set state to be state_1
    state = state_1

NameError: name 'model' is not defined

In [9]:
pygame.display.quit()